In [16]:
import polars as pl
import numpy as np
import random
from math import sin, cos, asin, sqrt, pi
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [17]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [18]:
num_cars = 100
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
#print(data.head())
print(data.collect())

shape: (632_325, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64        ┆ str           │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:03:02.026513 ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:30:00        ┆ 22.260341 ┆ -97.037515 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:35:55.764636 ┆ 22.260341 ┆ -97.037515 ┆ null          │
│ car_0  ┆ 2023-01-01T01:00:00        ┆ 22.254072 ┆ -97.029059 ┆ low_fuel      │
│ …      ┆ …                          ┆ …         ┆ …          ┆ …             │
│ car_99 ┆ 2023-01-07T23:35:31.648220 ┆ 22.540984 ┆ -91.205333 ┆ null          │
│ car_99

## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [19]:
def order_data(data):
    data = data.sort("car_id", "timestamp")
    return data
#es importate añadir ns para que todo quede en ell mismo timepo
data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime("ns")))

data = order_data(data)

print(data.collect())

shape: (632_325, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str    ┆ datetime[ns]               ┆ f64       ┆ f64        ┆ str           │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:03:02.026513 ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:30:00        ┆ 22.260341 ┆ -97.037515 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:35:55.764636 ┆ 22.260341 ┆ -97.037515 ┆ null          │
│ car_0  ┆ 2023-01-01 01:00:00        ┆ 22.254072 ┆ -97.029059 ┆ low_fuel      │
│ …      ┆ …                          ┆ …         ┆ …          ┆ …             │
│ car_99 ┆ 2023-01-07 23:39:41.457879 ┆ 22.540984 ┆ -91.205333 ┆ engine_check  │
│ car_99

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt


1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [20]:
data = (
    data.lazy()
    .sort(["car_id", "timestamp"])
    .with_columns(
        #en polars existe la funcion diff() que calcula la diferencia entre dos columnas
        #por default usa la row pasada y por ejmeplo diff(2) usa dos row atras
        pl.col("timestamp").diff().over("car_id").alias("notification_time")
    )
)

data = data.with_columns(
    pl.when(pl.col("notification_time").is_null())
    .then(pl.lit(None))
    .otherwise(pl.col("notification_time"))
    .alias("notification_time")
)
data = data.collect()

print(data)


shape: (632_325, 6)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┬───────────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  ┆ notification_time │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           ┆ ---               │
│ str    ┆ datetime[ns]               ┆ f64       ┆ f64        ┆ str           ┆ duration[ns]      │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╪═══════════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      ┆ null              │
│ car_0  ┆ 2023-01-01 00:03:02.026513 ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      ┆ 3m 2s 26513µs     │
│ car_0  ┆ 2023-01-01 00:30:00        ┆ 22.260341 ┆ -97.037515 ┆ engine_check  ┆ 26m 57s 973487µs  │
│ car_0  ┆ 2023-01-01 00:35:55.764636 ┆ 22.260341 ┆ -97.037515 ┆ null          ┆ 5m 55s 764636µs   │
│ car_0  ┆ 2023-01-01 01:00:00        ┆ 22.254072 ┆ -97.029059 ┆ low_fu

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [21]:
from math import radians, sin, cos, sqrt, atan2

# Formula de haversine que calcula la distancia entre dos puntos en la tierra
def haversine(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):
        return None

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_of_earth = 6371  # Radio de la tierra
    distance = radius_of_earth * c

    return distance

# Columnas intermedias para calcular distancia
#intente hacerlo sin estas columnas auxiliares como en el inciso anterior
#pero como no es una funcion integra de polars pude
data = data.with_columns(pl.col("latitude").shift().alias("latitude_shifted"))
data = data.with_columns(pl.col("longitude").shift().alias("longitude_shifted"))

data = data.with_columns(
    pl.when(
        #es importante que ninguna sea nula sino solo le ponemos el valor de nulo 
        pl.col("latitude").is_not_null() &
        pl.col("longitude").is_not_null() &
        pl.col("latitude_shifted").is_not_null() &
        pl.col("longitude_shifted").is_not_null()
    ).then(
        #usamos una funcion lambda para apllicar funciones no predeterminadas por polalrs
        pl.struct(["latitude", "longitude", "latitude_shifted", "longitude_shifted"]).apply(
            lambda row: haversine(row["latitude"], row["longitude"], row["latitude_shifted"], row["longitude_shifted"])
        )
    ).otherwise(pl.lit(None)).alias("distance")
)
#los usuarios facilemtne pueden leer la latitud/longitud anterior, no tiene sentido mantener estas columnas
data=data.drop("latitude_shifted", "longitude_shifted")
print(data)



/var/folders/rw/9dc4ss41167dpttmz9h6f1xr0000gn/T/ipykernel_18287/2158031673.py:37: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.struct(["latitude", "longitude", "latitude_shifted", "longitude_shifted"]).apply(
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


shape: (632_325, 7)
┌────────┬──────────────────┬───────────┬────────────┬───────────────┬──────────────────┬──────────┐
│ car_id ┆ timestamp        ┆ latitude  ┆ longitude  ┆ notification  ┆ notification_tim ┆ distance │
│ ---    ┆ ---              ┆ ---       ┆ ---        ┆ ---           ┆ e                ┆ ---      │
│ str    ┆ datetime[ns]     ┆ f64       ┆ f64        ┆ str           ┆ ---              ┆ f64      │
│        ┆                  ┆           ┆            ┆               ┆ duration[ns]     ┆          │
╞════════╪══════════════════╪═══════════╪════════════╪═══════════════╪══════════════════╪══════════╡
│ car_0  ┆ 2023-01-01       ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      ┆ null             ┆ null     │
│        ┆ 00:00:00         ┆           ┆            ┆               ┆                  ┆          │
│ car_0  ┆ 2023-01-01       ┆ 22.260871 ┆ -97.047137 ┆ low_fuel      ┆ 3m 2s 26513µs    ┆ 0.0      │
│        ┆ 00:03:02.026513  ┆           ┆            ┆               ┆ 

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars distanceespecificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de ``


In [22]:
interval = "5m" #intervalos de 5 mins

#usamos groupby_dynamic para agrupar los datos que vayan en itervalos no uniformes
result_notification = data.groupby_dynamic(
    by="car_id",
    index_column="timestamp",
    every=interval,
    period=interval,
    closed="both",
    include_boundaries=True
).agg([
    pl.col("notification_time").mean().alias("mean_time_ns"),
    pl.col("notification_time").median().alias("median_time_ns"),
    pl.col("notification_time").var().alias("variance_time_ns"),
    pl.col("notification_time").max().alias("max_time_ns"),
    pl.col("notification_time").min().alias("min_time_ns")
])

print(result_notification)

result_distance = data.groupby_dynamic(
    by="car_id",
    index_column="timestamp",
    every=interval,
    period=interval,
    closed="both",
    include_boundaries=True
).agg([
    pl.col("distance").mean().alias("mean_distance"),
    pl.col("distance").median().alias("median_distance"),
    pl.col("distance").var().alias("variance_distance"),
    pl.col("distance").max().alias("max_distance"),
    pl.col("distance").min().alias("min_distance")
])

print(result_distance)

/var/folders/rw/9dc4ss41167dpttmz9h6f1xr0000gn/T/ipykernel_18287/1011679719.py:4: DeprecationWarning: `groupby_dynamic` is deprecated. It has been renamed to `group_by_dynamic`.
  result_notification = data.groupby_dynamic(
/var/folders/rw/9dc4ss41167dpttmz9h6f1xr0000gn/T/ipykernel_18287/1011679719.py:11: DeprecationWarning: `truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.
  ).agg([


shape: (137_895, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ _lower_bou ┆ _upper_bou ┆ timestamp ┆ … ┆ median_ti ┆ variance_ ┆ max_time_ ┆ min_time_ │
│ ---    ┆ ndary      ┆ ndary      ┆ ---       ┆   ┆ me_ns     ┆ time_ns   ┆ ns        ┆ ns        │
│ str    ┆ ---        ┆ ---        ┆ datetime[ ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ datetime[n ┆ datetime[n ┆ ns]       ┆   ┆ duration[ ┆ duration[ ┆ duration[ ┆ duration[ │
│        ┆ s]         ┆ s]         ┆           ┆   ┆ ns]       ┆ ns]       ┆ ns]       ┆ ns]       │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2022-12-31 ┆ 2023-01-01 ┆ 2022-12-3 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆ 23:55:00   ┆ 00:00:00   ┆ 1         ┆   ┆           ┆           ┆           ┆           │
│        ┆            ┆            ┆ 23:55:00  ┆   ┆           ┆       

/var/folders/rw/9dc4ss41167dpttmz9h6f1xr0000gn/T/ipykernel_18287/1011679719.py:21: DeprecationWarning: `groupby_dynamic` is deprecated. It has been renamed to `group_by_dynamic`.
  result_distance = data.groupby_dynamic(
/var/folders/rw/9dc4ss41167dpttmz9h6f1xr0000gn/T/ipykernel_18287/1011679719.py:28: DeprecationWarning: `truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.
  ).agg([
